In [1]:
import pandas as pd
import numpy as np
import json
import urllib
import datetime
import glob
import os
import psycopg2
import psycopg2.extras as extras

In [2]:
# This notebook was last run on Jan 24, 2023. Recall that the initial ETL process pulled 2015-Aug 2022 data from the GIS web
# service to populate a local Postgres database. Initially, I "manually" ran the notebook to update the database with Sep, Oct
# and Nov 2022 data, running it separately for each month after updating the 'where' clause from the layers Siniestro,
# Con Herido and Con Muerto by manually entering the corresponding year and month, e.g.,
# 
# where_clause = "ANO_OCURRENCIA_ACC = 2022 AND MES_OCURRENCIA_ACC = 'SEPTIEMBRE'" 
#
# I then made use of the code from the cell below and updated the 'where' clause from the mentioned layers to update the
# database with Dec 2022 data

In [3]:
# When updating the database, the idea is to place ourselves in the current year and month (year yt and
# month mt = 1, 2, ..., 12) to update the database with data from year yt and month mt-1. The dictionary below reflects such a
# mapping, while the code that follows controls the case where the current month is Jan (mt = 1), which indicates we need to
# update the database with data from month Dec (mt = 12) and year yt-1.

# Given the time it takes for the Secretary of Mobility of Bogotá to update the GIS web service, we suggest to run this
# notebook halfway through the month

month_es = {
    "01":"DICIEMBRE",
    "02":"ENERO",
    "03":"FEBRERO",
    "04":"MARZO",
    "05":"ABRIL",
    "06":"MAYO",
    "07":"JUNIO",
    "08":"JULIO",
    "09":"AGOSTO",
    "10":"SEPTIEMBRE",
    "11":"OCTUBRE",
    "12":"NOVIEMBRE",
}

today = datetime.date.today()
year_today = today.year
month_today = today.strftime("%m")

year_query = year_today
month_query = month_es.get(month_today)

if month_today == "01":
    year_query = year_query - 1

In [5]:
##################################################
##### Accidents #####
##################################################

##### PULLING DATA FROM GIS WEB SERVICE

# We specify the URL
url = "https://sig.simur.gov.co/arcgis/rest/services/Accidentalidad/WSAcidentalidad_Publico/FeatureServer/0/query?"

where_clause = "ANO_OCURRENCIA_ACC = " + str(year_query) + " AND MES_OCURRENCIA_ACC = '" + month_query + "'"
accidents = {'where': where_clause,
    'outFields': 'OBJECTID, FORMULARIO, LOCALIDAD, CIV, PK_CALZADA, CLASE_ACC, GRAVEDAD, FECHA_HORA_ACC',
    'returnGeometry': 'true',      
    'f': 'json',
}
encode_accidents = urllib.parse.urlencode(accidents).encode("utf-8")

# We create a request and read it using urllib
response_accidents = urllib.request.urlopen(url, encode_accidents)
query_accidents = response_accidents.read()

# We write the JSON response to a file
with open("pipeline_accidents.json", "wb") as json_file:
    json_file.write(query_accidents)
# We load the JSON file into a DataFrame 
with open("pipeline_accidents.json", "r") as f:
    accidents_df = pd.json_normalize(json.loads(f.read()), "features")

# We delete the JSON file
    # https://linuxize.com/post/python-delete-files-and-directories/
os.unlink("pipeline_accidents.json")

# We rename the columns by removing "attributes." from their names
column_dict = {} 
for i in range(len(accidents_df.columns)):
    # Map the old column names with the new ones (without "attributes.") and store the mapping in a dictionary
    column_dict.update({accidents_df.columns[i]: accidents_df.columns[i].replace("attributes.", "")})

# We rename the columns using the dictionary
accidents_df.rename(columns = column_dict, inplace = True)

##### DATA PREPARATION

# All data preparation of this and the remaining tables is done based on the data preparation performed during the initial
# ETL process (see the notebooks 3_data_preparation and 4_database_creation from the folder initial_etl)

# Extract date info
    # The unix time is UTC time, which is five hours ahead of Bogotá
    # https://thispointer.com/subtract-hours-from-datetime-in-python/
accidents_df["FECHA_HORA_ACC_r"] = pd.to_datetime(accidents_df["FECHA_HORA_ACC"], unit = "ms") - pd.DateOffset(hours = 5)

# The following approach is more formal, but leaves the column with a final "flag" indicating the time difference
# w.r.t. UTC
#accidents_df["FECHA_HORA_ACC_r"] = pd.to_datetime(accidents_df["FECHA_HORA_ACC"], unit = "ms").dt.tz_localize("UTC").\
#    dt.tz_convert("America/Bogota")

accidents_df["ANO_OCURRENCIA_ACC"] = accidents_df["FECHA_HORA_ACC_r"].dt.year

# Install the Spanish locale before running the following code
    # https://serverpilot.io/docs/how-to-install-locales/
accidents_df["MES_OCURRENCIA_ACC"] = accidents_df["FECHA_HORA_ACC_r"].dt.month_name(locale = "es_ES.UTF-8")
accidents_df["MES_NRO_OCURRENCIA_ACC"] = accidents_df["FECHA_HORA_ACC_r"].dt.month
accidents_df["DIA_OCURRENCIA_ACC"] = accidents_df["FECHA_HORA_ACC_r"].dt.day_name(locale = "es_ES.UTF-8")

# 1 is Monday, 2 is Tuesday and so on
accidents_df["DIA_NRO_OCURRENCIA_ACC"] = accidents_df["FECHA_HORA_ACC_r"].dt.dayofweek + 1

accidents_df["DIA_MES_OCURRENCIA_ACC"] = accidents_df["FECHA_HORA_ACC_r"].dt.day
accidents_df["HORA_OCURRENCIA_ACC"] = accidents_df["FECHA_HORA_ACC_r"].dt.hour

# Uppercase month and day of week to keep consistency with original format
accidents_df["MES_OCURRENCIA_ACC"] = accidents_df["MES_OCURRENCIA_ACC"].str.upper()
accidents_df["DIA_OCURRENCIA_ACC"] = accidents_df["DIA_OCURRENCIA_ACC"].str.upper()

# Rename the longitude and latitude columns
accidents_df.rename(columns = {"geometry.x": "LONGITUDE", "geometry.y": "LATITUDE"}, inplace = True)

# CIV and PK_CALZADA NaN treatment: fill with 0s and then change their type to int
accidents_df["CIV"] = accidents_df["CIV"].fillna(0)
accidents_df["PK_CALZADA"] = accidents_df["PK_CALZADA"].fillna(0)
accidents_df["CIV"] = accidents_df["CIV"].astype(int)
accidents_df["PK_CALZADA"] = accidents_df["PK_CALZADA"].astype(int)

# Make some final adjustments
accidents_df["FECHA_ACC"] = pd.to_datetime(accidents_df["FECHA_HORA_ACC_r"], unit = "ms").dt.date
accidents_df.drop({"FECHA_HORA_ACC", "FECHA_HORA_ACC_r"}, axis = 1, inplace = True)
accidents_df.rename(columns = {"LONGITUDE": "LONGITUD", "LATITUDE": "LATITUD"}, inplace = True)
accidents_df.head()

,OBJECTID,FORMULARIO,LOCALIDAD,CIV,PK_CALZADA,CLASE_ACC,GRAVEDAD,LONGITUD,LATITUD,ANO_OCURRENCIA_ACC,MES_OCURRENCIA_ACC,MES_NRO_OCURRENCIA_ACC,DIA_OCURRENCIA_ACC,DIA_NRO_OCURRENCIA_ACC,DIA_MES_OCURRENCIA_ACC,HORA_OCURRENCIA_ACC,FECHA_ACC
0,453101,A001519131,CIUDAD BOLIVAR,19007302,0,CHOQUE,CON HERIDOS,-74.138204,4.557293,2023,ENERO,1,LUNES,1,2,11,2023-01-02
1,453102,A001516348,ANTONIO NARINO,15000376,0,CHOQUE,CON HERIDOS,-74.115338,4.592395,2023,ENERO,1,LUNES,1,2,10,2023-01-02
2,453104,A001519104,BOSA,7006556,0,CHOQUE,CON HERIDOS,-74.189906,4.603582,2023,ENERO,1,LUNES,1,2,23,2023-01-02
3,453107,A001518989,SUBA,50007697,0,ATROPELLO,CON HERIDOS,-74.096169,4.746282,2023,ENERO,1,MARTES,2,3,15,2023-01-03
4,453108,A001518963,CHAPINERO,2001760,0,CHOQUE,CON HERIDOS,-74.066455,4.638471,2023,ENERO,1,MARTES,2,3,0,2023-01-03


In [6]:
##################################################
##### Injured people #####
##################################################

##### PULLING DATA FROM GIS WEB SERVICE

# We specify the URL
url = "https://sig.simur.gov.co/arcgis/rest/services/Accidentalidad/WSAcidentalidad_Publico/FeatureServer/2/query?"

where_clause = "ANO_OCURRENCIA_ACC = " + str(year_query) + " AND MES_OCURRENCIA_ACC = '" + month_query + "'"
injured_people = {'where': where_clause,
    'outFields': 'OBJECTID, FORMULARIO, LOCALIDAD, CLASE_ACC, CONDICION, GENERO, EDAD, FECHA_HORA_ACC',
    'returnGeometry': 'true',      
    'f': 'json',
}
encode_injured_people = urllib.parse.urlencode(injured_people).encode("utf-8")

# We create a request and read it using urllib
response_injured_people = urllib.request.urlopen(url, encode_injured_people)
query_injured_people = response_injured_people.read()

# We write the JSON response to a file
with open("pipeline_injured_people.json", "wb") as json_file:
    json_file.write(query_injured_people)
# We load the JSON file into a DataFrame 
with open("pipeline_injured_people.json", "r") as f:
    injured_people_df = pd.json_normalize(json.loads(f.read()), "features")

# We delete the JSON file
    # https://linuxize.com/post/python-delete-files-and-directories/
os.unlink("pipeline_injured_people.json")

# We rename the columns by removing "attributes." from their names
column_dict = {} 
for i in range(len(injured_people_df.columns)):
    # Map the old column names with the new ones (without "attributes.") and store the mapping in a dictionary
    column_dict.update({injured_people_df.columns[i]: injured_people_df.columns[i].replace("attributes.", "")})

# We rename the columns using the dictionary
injured_people_df.rename(columns = column_dict, inplace = True)

##### DATA PREPARATION

# Extract date info
    # The unix time is UTC time, which is five hours ahead of Bogotá
    # https://thispointer.com/subtract-hours-from-datetime-in-python/
injured_people_df["FECHA_HORA_ACC_r"] = pd.to_datetime(injured_people_df["FECHA_HORA_ACC"], unit = "ms") - pd.DateOffset(hours = 5)

# The following approach is more formal, but leaves the column with a final "flag" indicating the time difference
# w.r.t. UTC
#injured_people_df["FECHA_HORA_ACC_r"] = pd.to_datetime(injured_people_df["FECHA_HORA_ACC"], unit = "ms").dt.tz_localize("UTC").\
#    dt.tz_convert("America/Bogota")

injured_people_df["ANO_OCURRENCIA_ACC"] = injured_people_df["FECHA_HORA_ACC_r"].dt.year

# Install the Spanish locale before running the following code
    # https://serverpilot.io/docs/how-to-install-locales/
injured_people_df["MES_OCURRENCIA_ACC"] = injured_people_df["FECHA_HORA_ACC_r"].dt.month_name(locale = "es_ES.UTF-8")
injured_people_df["MES_NRO_OCURRENCIA_ACC"] = injured_people_df["FECHA_HORA_ACC_r"].dt.month
injured_people_df["DIA_OCURRENCIA_ACC"] = injured_people_df["FECHA_HORA_ACC_r"].dt.day_name(locale = "es_ES.UTF-8")

# 1 is Monday, 2 is Tuesday and so on
injured_people_df["DIA_NRO_OCURRENCIA_ACC"] = injured_people_df["FECHA_HORA_ACC_r"].dt.dayofweek + 1

injured_people_df["DIA_MES_OCURRENCIA_ACC"] = injured_people_df["FECHA_HORA_ACC_r"].dt.day
injured_people_df["HORA_OCURRENCIA_ACC"] = injured_people_df["FECHA_HORA_ACC_r"].dt.hour

# Uppercase month and day of week to keep consistency with original format
injured_people_df["MES_OCURRENCIA_ACC"] = injured_people_df["MES_OCURRENCIA_ACC"].str.upper()
injured_people_df["DIA_OCURRENCIA_ACC"] = injured_people_df["DIA_OCURRENCIA_ACC"].str.upper()

# Rename the longitude and latitude columns
injured_people_df.rename(columns = {"geometry.x": "LONGITUDE", "geometry.y": "LATITUDE"}, inplace = True)

# GENERO NaN treatment: fill with "SIN INFORMACION"
injured_people_df["GENERO"] = injured_people_df["GENERO"].fillna("SIN INFORMACION")

# EDAD NaN treatment: since we don't have information on what a zero represents (is it a baby that hasn't turned one year yet or
# a null value?), we leave this feature as it is

# Make some final adjustments
injured_people_df["FECHA_ACC"] = pd.to_datetime(injured_people_df["FECHA_HORA_ACC_r"], unit = "ms").dt.date
injured_people_df.drop({"FECHA_HORA_ACC", "FECHA_HORA_ACC_r"}, axis = 1, inplace = True)
injured_people_df.rename(columns = {"LONGITUDE": "LONGITUD", "LATITUDE": "LATITUD"}, inplace = True)
injured_people_df.head()

,OBJECTID,FORMULARIO,LOCALIDAD,CLASE_ACC,CONDICION,GENERO,EDAD,LONGITUD,LATITUD,ANO_OCURRENCIA_ACC,MES_OCURRENCIA_ACC,MES_NRO_OCURRENCIA_ACC,DIA_OCURRENCIA_ACC,DIA_NRO_OCURRENCIA_ACC,DIA_MES_OCURRENCIA_ACC,HORA_OCURRENCIA_ACC,FECHA_ACC
0,1425901,A001518356,SAN CRISTOBAL,CHOQUE,MOTOCICLISTA,MASCULINO,20.0,-74.094279,4.567116,2023,ENERO,1,DOMINGO,7,1,8,2023-01-01
1,1425848,A001518554,BOSA,CHOQUE,MOTOCICLISTA,MASCULINO,23.0,-74.208053,4.611807,2023,ENERO,1,DOMINGO,7,1,12,2023-01-01
2,1429388,A001519079,SUBA,ATROPELLO,PEATON,FEMENINO,50.0,-74.084957,4.726734,2023,ENERO,1,DOMINGO,7,1,2,2023-01-01
3,1429387,A001519079,SUBA,ATROPELLO,PEATON,MASCULINO,43.0,-74.084957,4.726734,2023,ENERO,1,DOMINGO,7,1,2,2023-01-01
4,1425833,A001518544,SUBA,CHOQUE,CONDUCTOR,MASCULINO,70.0,-74.094743,4.720117,2023,ENERO,1,DOMINGO,7,1,7,2023-01-01


In [7]:
##################################################
##### Killed people #####
##################################################

##### PULLING DATA FROM GIS WEB SERVICE

# We specify the URL
url = "https://sig.simur.gov.co/arcgis/rest/services/Accidentalidad/WSAcidentalidad_Publico/FeatureServer/1/query?"

where_clause = "ANO_OCURRENCIA_ACC = " + str(year_query) + " AND MES_OCURRENCIA_ACC = '" + month_query + "'"
killed_people = {'where': where_clause,
    'outFields': 'OBJECTID, FORMULARIO, LOCALIDAD, CLASE_ACC, CONDICION, GENERO, EDAD, MUERTE_POSTERIOR, FECHA_POSTERIOR_MUERTE, FECHA_HORA_ACC',
    'returnGeometry': 'true',      
    'f': 'json',
}
encode_killed_people = urllib.parse.urlencode(killed_people).encode("utf-8")

# We create a request and read it using urllib
response_killed_people = urllib.request.urlopen(url, encode_killed_people)
query_killed_people = response_killed_people.read()

# We write the JSON response to a file
with open("pipeline_killed_people.json", "wb") as json_file:
    json_file.write(query_killed_people)
# We load the JSON file into a DataFrame 
with open("pipeline_killed_people.json", "r") as f:
    killed_people_df = pd.json_normalize(json.loads(f.read()), "features")

# We delete the JSON file
    # https://linuxize.com/post/python-delete-files-and-directories/
os.unlink("pipeline_killed_people.json")

# We rename the columns by removing "attributes." from their names
column_dict = {} 
for i in range(len(killed_people_df.columns)):
    # Map the old column names with the new ones (without "attributes.") and store the mapping in a dictionary
    column_dict.update({killed_people_df.columns[i]: killed_people_df.columns[i].replace("attributes.", "")})

# We rename the columns using the dictionary
killed_people_df.rename(columns = column_dict, inplace = True)

##### DATA PREPARATION

# Extract date info
    # The unix time is UTC time, which is five hours ahead of Bogotá
    # https://thispointer.com/subtract-hours-from-datetime-in-python/
killed_people_df["FECHA_HORA_ACC_r"] = pd.to_datetime(killed_people_df["FECHA_HORA_ACC"], unit = "ms") - pd.DateOffset(hours = 5)

# The following approach is more formal, but leaves the column with a final "flag" indicating the time difference
# w.r.t. UTC
#killed_people_df["FECHA_HORA_ACC_r"] = pd.to_datetime(killed_people_df["FECHA_HORA_ACC"], unit = "ms").dt.tz_localize("UTC").\
#    dt.tz_convert("America/Bogota")

killed_people_df["ANO_OCURRENCIA_ACC"] = killed_people_df["FECHA_HORA_ACC_r"].dt.year

# Install the Spanish locale before running the following code
    # https://serverpilot.io/docs/how-to-install-locales/
killed_people_df["MES_OCURRENCIA_ACC"] = killed_people_df["FECHA_HORA_ACC_r"].dt.month_name(locale = "es_ES.UTF-8")
killed_people_df["MES_NRO_OCURRENCIA_ACC"] = killed_people_df["FECHA_HORA_ACC_r"].dt.month
killed_people_df["DIA_OCURRENCIA_ACC"] = killed_people_df["FECHA_HORA_ACC_r"].dt.day_name(locale = "es_ES.UTF-8")

# 1 is Monday, 2 is Tuesday and so on
killed_people_df["DIA_NRO_OCURRENCIA_ACC"] = killed_people_df["FECHA_HORA_ACC_r"].dt.dayofweek + 1

killed_people_df["DIA_MES_OCURRENCIA_ACC"] = killed_people_df["FECHA_HORA_ACC_r"].dt.day
killed_people_df["HORA_OCURRENCIA_ACC"] = killed_people_df["FECHA_HORA_ACC_r"].dt.hour

# Uppercase month and day of week to keep consistency with original format
killed_people_df["MES_OCURRENCIA_ACC"] = killed_people_df["MES_OCURRENCIA_ACC"].str.upper()
killed_people_df["DIA_OCURRENCIA_ACC"] = killed_people_df["DIA_OCURRENCIA_ACC"].str.upper()

# Rename the longitude and latitude columns
killed_people_df.rename(columns = {"geometry.x": "LONGITUDE", "geometry.y": "LATITUDE"}, inplace = True)

# GENERO NaN treatment: fill with "SIN INFORMACION"
killed_people_df["GENERO"] = killed_people_df["GENERO"].fillna("SIN INFORMACION")

# MUERTE_POSTERIOR NaN treatment: fill with "N" (the person die in the accident, not after)
killed_people_df["MUERTE_POSTERIOR"] = killed_people_df["MUERTE_POSTERIOR"].fillna("N")

# EDAD NaN treatment: since we don't have information on what a zero represents (is it a baby that hasn't turned one year yet or
# a null value?), we leave this feature as it is

# Make some final adjustments
killed_people_df["FECHA_ACC"] = pd.to_datetime(killed_people_df["FECHA_HORA_ACC_r"], unit = "ms").dt.date
killed_people_df.drop({"FECHA_HORA_ACC", "FECHA_HORA_ACC_r"}, axis = 1, inplace = True)
killed_people_df.rename(columns = {"LONGITUDE": "LONGITUD", "LATITUDE": "LATITUD"}, inplace = True)
killed_people_df.head()

,OBJECTID,FORMULARIO,LOCALIDAD,CLASE_ACC,CONDICION,GENERO,EDAD,MUERTE_POSTERIOR,FECHA_POSTERIOR_MUERTE,LONGITUD,LATITUD,ANO_OCURRENCIA_ACC,MES_OCURRENCIA_ACC,MES_NRO_OCURRENCIA_ACC,DIA_OCURRENCIA_ACC,DIA_NRO_OCURRENCIA_ACC,DIA_MES_OCURRENCIA_ACC,HORA_OCURRENCIA_ACC,FECHA_ACC
0,1414893,A001519811,KENNEDY,ATROPELLO,PEATON,FEMENINO,23.0,N,NaN,-74.146904,4.618255,2023,ENERO,1,DOMINGO,7,22,19,2023-01-22
1,1417333,A001518441,BOSA,VOLCAMIENTO,MOTOCICLISTA,MASCULINO,44.0,N,NaN,-74.167467,4.595989,2023,ENERO,1,SÁBADO,6,21,15,2023-01-21
2,1415854,A001519199,BOSA,CHOQUE,MOTOCICLISTA,MASCULINO,44.0,N,NaN,-74.189920,4.621967,2023,ENERO,1,MIÉRCOLES,3,18,23,2023-01-18
3,1424492,A001519962,ANTONIO NARINO,CHOQUE,MOTOCICLISTA,MASCULINO,25.0,N,NaN,-74.102841,4.586277,2023,ENERO,1,JUEVES,4,26,21,2023-01-26
4,1423628,A001518935,KENNEDY,ATROPELLO,PEATON,MASCULINO,25.0,N,NaN,-74.137951,4.627230,2023,ENERO,1,LUNES,1,2,3,2023-01-02


In [8]:
##################################################
##### Causes #####
##################################################

##### PULLING DATA FROM GIS WEB SERVICE

# Given the lack of date information in the remaining layers of the GIS web service (Causa, Actor Vial and Vehiculo) and the
# impossibility of executing JOIN operations in the service, we must rely on the FORMULARIO field from accidents_df to
# retrieve the information from these layers

formulario_series = accidents_df["FORMULARIO"].copy()

# The 'where' clause only allows up to 1.000 values inside IN (), so we need to loop accordingly. We have to execute a query n
# times, each time but the last one with 1.000 FORMULARIO values inside IN (). In the last iteration, the number of FORMULARIO
# values will depend on the size of formulario_series  

# The following code creates the indexes for each query iteration. It controls whether the quantity of FORMULARIO values is 
# greater than 999 so that we can properly create the indexes and loop accordingly when creating the JSON files
indexes = []

if len(formulario_series) >= 1000:
    formulario_loop = int(len(formulario_series)/1000)
    for i in range(formulario_loop):
        indexes.append([i*1000, i*1000 + 1000])
    indexes.append([(i + 1)*1000, len(formulario_series)])
else:
    formulario_loop = 0
    indexes.append([0, len(formulario_series)])   

# We specify the URL
url = "https://sig.simur.gov.co/arcgis/rest/services/Accidentalidad/WSAcidentalidad_Publico/FeatureServer/5/query?"

file_name_init = "pipeline_causes"
file_name_end = ".json"

# We create a JSON file for each query iteration
for i in range(formulario_loop + 1):
    
    # We specify the query
    where_clause = "FORMULARIO IN ("
    
    # We append the FORMULARIO values to the 'where' clause
    formulario_series_loop = formulario_series[indexes[i][0]:indexes[i][1]].copy()
    for formulario in formulario_series_loop:
        where_clause += "'" + formulario + "', "
    where_clause = where_clause[:len(where_clause)-2] + ")" # After this loop, where_clause ends with ", ", so we replace this
        # with ")" 
    
    causes = {'where': where_clause,
        'outFields': 'OBJECTID, FORMULARIO, CODIGO_VEHICULO, CODIGO_CAUSA, NOMBRE, TIPO, TIPO_CAUSA',
        'returnGeometry': 'false',      
        'f': 'json',
    }
    encode_causes = urllib.parse.urlencode(causes).encode("utf-8")
    
    # We create a request and read it using urllib
    response_causes = urllib.request.urlopen(url, encode_causes)
    query_causes = response_causes.read()
    
    # We write the JSON response to a file
    file_name = file_name_init + str(i + 1) + file_name_end
    with open(file_name, "wb") as json_file:
        json_file.write(query_causes)
    
    print("Iter: " + str(i)) # Keep track of the process (we know how many files we're creating)

df_list = [] # List to store DataFrames generated from JSON files

# Retrieve list of JSON files
json_pattern = os.path.join("*.json")
file_list = glob.glob(json_pattern)

for file in file_list:
    # Read a JSON file
    with open(file, "r") as f:
        data = json.loads(f.read())
    
    # Load the JSON file into a DataFrame and store the DataFrame in a list of DataFrames
        # https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8. Check Section 3
    df_list.append(pd.json_normalize(data, "features"))

# We delete the JSON files
    # https://linuxize.com/post/python-delete-files-and-directories/
files = glob.glob("*.json")
for f in files:
    os.unlink(f)

# Concatenate all DataFrames from the list into a single DataFrame
causes_df = pd.concat(df_list, axis = 0)

column_dict = {} # Dictionary to store old and new column names for the concatenated DataFrame

# We rename the columns by removing "attributes." from their names
for i in range(len(causes_df.columns)):
    # Map the old column names with the new ones (without "attributes.") and store the mapping in a dictionary
    column_dict.update({causes_df.columns[i]: causes_df.columns[i].replace("attributes.", "")})

# We rename the columns using the dictionary
causes_df.rename(columns = column_dict, inplace = True)

# In case we had worked with several JSON files, there could be repeated indexes after concatenating the DataFrames. We take
# care of this by resetting the index and dropping the generated index column
causes_df.reset_index(inplace = True)
causes_df.drop("index", axis = 1, inplace = True)

##### DATA PREPARATION

# Fill potential NaN values in TIPO and TIPO_CAUSA
causes_df.fillna("SIN ESTABLECER", inplace = True)

# Do some cleaning in rows with values "OTRAS"
causes_df["NOMBRE"] = np.where(causes_df["NOMBRE"] == "OTRAS", "OTRA", causes_df["NOMBRE"])
causes_df.head()

Iter: 0
Iter: 1


,OBJECTID,FORMULARIO,CODIGO_VEHICULO,CODIGO_CAUSA,NOMBRE,TIPO,TIPO_CAUSA
0,381090,A001513685,2.0,122,NO UTILIZAR - GIRAR BRUSCAMENTE,CG,CONDUCTOR
1,381091,A001513685,2.0,157,OTRA,CG,CONDUCTOR
2,375129,A001514310,1.0,143,PONER EN MARCHA UN VEHICULO SIN PRECAUCIONES,CG,CONDUCTOR
3,377143,A001514317,1.0,104,ADELANTAR INVADIENDO VIA,CG,CONDUCTOR
4,377144,A001514317,2.0,122,NO UTILIZAR - GIRAR BRUSCAMENTE,CG,CONDUCTOR


In [9]:
##################################################
##### Actors #####
##################################################

# PULLING DATA FROM GIS WEB SERVICE

formulario_series = accidents_df["FORMULARIO"].copy()
indexes = []

if len(formulario_series) >= 1000:
    formulario_loop = int(len(formulario_series)/1000)
    for i in range(formulario_loop):
        indexes.append([i*1000, i*1000 + 1000])
    indexes.append([(i + 1)*1000, len(formulario_series)])
else:
    formulario_loop = 0
    indexes.append([0, len(formulario_series)])   

# We specify the URL
url = "https://sig.simur.gov.co/arcgis/rest/services/Accidentalidad/WSAcidentalidad_Publico/FeatureServer/6/query?"

file_name_init = "pipeline_actors"
file_name_end = ".json"

# We create a JSON file for each query iteration
for i in range(formulario_loop + 1):
    
    # We specify the query
    where_clause = "FORMULARIO IN ("
    
    # We append the FORMULARIO values to the 'where' clause
    formulario_series_loop = formulario_series[indexes[i][0]:indexes[i][1]].copy()
    for formulario in formulario_series_loop:
        where_clause += "'" + formulario + "', "
    where_clause = where_clause[:len(where_clause)-2] + ")" # After this loop, where_clause ends with ", ", so we replace this
        # with ")" 
    
    actors = {'where': where_clause,
        'outFields': 'OBJECTID, FORMULARIO, CODIGO_VICTIMA, CODIGO_VEHICULO, CONDICION, GENERO, EDAD, ESTADO, MUERTE_POSTERIOR, FECHA_POSTERIOR_MUERTE',
        'returnGeometry': 'false',      
        'f': 'json',
    }
    encode_actors = urllib.parse.urlencode(actors).encode("utf-8")
    
    # We create a request and read it using urllib
    response_actors = urllib.request.urlopen(url, encode_actors)
    query_actors = response_actors.read()
    
    # We write the JSON response to a file
    file_name = file_name_init + str(i + 1) + file_name_end
    with open(file_name, "wb") as json_file:
        json_file.write(query_actors)
    
    print("Iter: " + str(i)) # Keep track of the process (we know how many files we're creating)

df_list = [] # List to store DataFrames generated from JSON files

# Retrieve list of JSON files
json_pattern = os.path.join("*.json")
file_list = glob.glob(json_pattern)

for file in file_list:
    # Read a JSON file
    with open(file, "r") as f:
        data = json.loads(f.read())
    
    # Load the JSON file into a DataFrame and store the DataFrame in a list of DataFrames
        # https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8. Check Section 3
    df_list.append(pd.json_normalize(data, "features"))

# We delete the JSON files
    # https://linuxize.com/post/python-delete-files-and-directories/
files = glob.glob("*.json")
for f in files:
    os.unlink(f)

# Concatenate all DataFrames from the list into a single DataFrame
actors_df = pd.concat(df_list, axis = 0)

column_dict = {} # Dictionary to store old and new column names for the concatenated DataFrame

# We rename the columns by removing "attributes." from their names
for i in range(len(actors_df.columns)):
    # Map the old column names with the new ones (without "attributes.") and store the mapping in a dictionary
    column_dict.update({actors_df.columns[i]: actors_df.columns[i].replace("attributes.", "")})

# We rename the columns using the dictionary
actors_df.rename(columns = column_dict, inplace = True)

# In case we had worked with several JSON files, there could be repeated indexes after concatenating the DataFrames. We take
# care of this by resetting the index and dropping the generated index column
actors_df.reset_index(inplace = True)
actors_df.drop("index", axis = 1, inplace = True)

##### DATA PREPARATION

# GENERO NaN treatment: fill with "SIN INFORMACION"
actors_df["GENERO"] = actors_df["GENERO"].fillna("SIN INFORMACION")

# EDAD NaN treatment: since we don't have information on what a zero represents (is it a baby that hasn't turned one year yet or
# a null value?), we leave this feature as it is

# CODIGO_VEHICULO and CONDICION NaN treatment: PASAJERO, PEATON and null values in CONDICION are associated with null values in
# CODIGO_VEHICULO. We fill null values in CONDICION with "SIN INFORMACION" and null values in CODIGO_VEHICULO with 0s and then
# change their type to int
actors_df["CONDICION"] = actors_df["CONDICION"].fillna("SIN INFORMACION")
actors_df["CODIGO_VEHICULO"] = actors_df["CODIGO_VEHICULO"].fillna(0)
actors_df["CODIGO_VEHICULO"] = actors_df["CODIGO_VEHICULO"].astype(int) 

# ESTADO NaN treatment: fill with "ILESO"
actors_df["ESTADO"] = actors_df["ESTADO"].fillna("ILESO")

# Based on the initial ETL process, we engineer a new feature called ESTADO_FINAL
actors_df["ESTADO_FINAL"] = np.where((actors_df["ESTADO"] == "MUERTO") | (actors_df["MUERTE_POSTERIOR"] == "S"), "MUERTO", \
    actors_df["ESTADO"])
actors_df.head()

Iter: 0
Iter: 1


,OBJECTID,FORMULARIO,CODIGO_VICTIMA,CODIGO_VEHICULO,CONDICION,GENERO,EDAD,ESTADO,MUERTE_POSTERIOR,FECHA_POSTERIOR_MUERTE,ESTADO_FINAL
0,1430268,A001513685,6.0,1,PASAJERO,MASCULINO,41.0,HERIDO,N,NaN,HERIDO
1,1430269,A001513685,5.0,1,PASAJERO,MASCULINO,20.0,HERIDO,N,NaN,HERIDO
2,1430270,A001513685,4.0,1,PASAJERO,FEMENINO,46.0,HERIDO,N,NaN,HERIDO
3,1430271,A001513685,3.0,1,PASAJERO,FEMENINO,25.0,HERIDO,N,NaN,HERIDO
4,1430272,A001513685,2.0,1,PASAJERO,FEMENINO,40.0,HERIDO,N,NaN,HERIDO


In [10]:
##################################################
##### Vehicles #####
##################################################

# PULLING DATA FROM GIS WEB SERVICE

formulario_series = accidents_df["FORMULARIO"].copy()
indexes = []

if len(formulario_series) >= 1000:
    formulario_loop = int(len(formulario_series)/1000)
    for i in range(formulario_loop):
        indexes.append([i*1000, i*1000 + 1000])
    indexes.append([(i + 1)*1000, len(formulario_series)])
else:
    formulario_loop = 0
    indexes.append([0, len(formulario_series)])

# We specify the URL
url = "https://sig.simur.gov.co/arcgis/rest/services/Accidentalidad/WSAcidentalidad_Publico/FeatureServer/4/query?"

file_name_init = "pipeline_vehicles"
file_name_end = ".json"

# We create a JSON file for each query iteration
for i in range(formulario_loop + 1):
    
    # We specify the query
    where_clause = "FORMULARIO IN ("
    
    # We append the FORMULARIO values to the 'where' clause
    formulario_series_loop = formulario_series[indexes[i][0]:indexes[i][1]].copy()
    for formulario in formulario_series_loop:
        where_clause += "'" + formulario + "', "
    where_clause = where_clause[:len(where_clause)-2] + ")" # After this loop, where_clause ends with ", ", so we replace this
        # with ")" 
    
    vehicles = {'where': where_clause,
        'outFields': 'OBJECTID, FORMULARIO, CODIGO_VEHICULO, CLASE, SERVICIO, MODALIDAD, ENFUGA',
        'returnGeometry': 'false',      
        'f': 'json',
    }
    encode_vehicles = urllib.parse.urlencode(vehicles).encode("utf-8")
    
    # We create a request and read it using urllib
    response_vehicles = urllib.request.urlopen(url, encode_vehicles)
    query_vehicles = response_vehicles.read()
    
    # We write the JSON response to a file
    file_name = file_name_init + str(i + 1) + file_name_end
    with open(file_name, "wb") as json_file:
        json_file.write(query_vehicles)
    
    print("Iter: " + str(i)) # Keep track of the process (we know how many files we're creating)

df_list = [] # List to store DataFrames generated from JSON files

# Retrieve list of JSON files
json_pattern = os.path.join("*.json")
file_list = glob.glob(json_pattern)

for file in file_list:
    # Read a JSON file
    with open(file, "r") as f:
        data = json.loads(f.read())
    
    # Load the JSON file into a DataFrame and store the DataFrame in a list of DataFrames
        # https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8. Check Section 3
    df_list.append(pd.json_normalize(data, "features"))

# We delete the JSON files
    # https://linuxize.com/post/python-delete-files-and-directories/
files = glob.glob("*.json")
for f in files:
    os.unlink(f)

# Concatenate all DataFrames from the list into a single DataFrame
vehicles_df = pd.concat(df_list, axis = 0)

column_dict = {} # Dictionary to store old and new column names for the concatenated DataFrame

# We rename the columns by removing "attributes." from their names
for i in range(len(vehicles_df.columns)):
    # Map the old column names with the new ones (without "attributes.") and store the mapping in a dictionary
    column_dict.update({vehicles_df.columns[i]: vehicles_df.columns[i].replace("attributes.", "")})

# We rename the columns using the dictionary
vehicles_df.rename(columns = column_dict, inplace = True)

# In case we had worked with several JSON files, there could be repeated indexes after concatenating the DataFrames. We take
# care of this by resetting the index and dropping the generated index column
vehicles_df.reset_index(inplace = True)
vehicles_df.drop("index", axis = 1, inplace = True)

##### DATA PREPARATION

# CODIGO_VEHICULO NaN treatment: fill with 1s and then change its type to int
vehicles_df["CODIGO_VEHICULO"] = vehicles_df["CODIGO_VEHICULO"].fillna(1)
vehicles_df["CODIGO_VEHICULO"] = vehicles_df["CODIGO_VEHICULO"].astype(int)

# CLASE NaN treatment: fill with "SIN INFORMACION"
vehicles_df["CLASE"] = vehicles_df["CLASE"].fillna("SIN INFORMACION")

# SERVICIO NaN treatment: fill with "SIN INFORMACION"
# Do some cleaning in rows with values " SIN INFORMACION"
vehicles_df["SERVICIO"] = vehicles_df["SERVICIO"].fillna("SIN INFORMACION")
vehicles_df["SERVICIO"] = np.where(vehicles_df["SERVICIO"] == " SIN INFORMACION", "SIN INFORMACION", vehicles_df["SERVICIO"])

# MODALIDAD NaN treatment: fill with "SIN INFORMACION".
# While this could be done in a single step, we first checked with SERVICIO PUBLICO since the understanding was that MODALIDAD
# was a feature that only applied to SERVICIO PUBLICO vehicles
vehicles_df["MODALIDAD"] = np.where((vehicles_df["SERVICIO"] == "PUBLICO") & (vehicles_df["MODALIDAD"].isna()), \
    "SIN INFORMACION", vehicles_df["MODALIDAD"])
vehicles_df["MODALIDAD"] = np.where((vehicles_df["SERVICIO"] != "PUBLICO") & (vehicles_df["MODALIDAD"].isna()), \
    "SIN INFORMACION", vehicles_df["MODALIDAD"])

# Based on the initial ETL process, the ENFUGA NaN values are not treated since the info is rather confusing
vehicles_df.head()

Iter: 0
Iter: 1


,OBJECTID,FORMULARIO,CODIGO_VEHICULO,CLASE,SERVICIO,MODALIDAD,ENFUGA
0,14284207,A001513685,1,BUS,PUBLICO,PASAJEROS - MASIVO,N
1,14284208,A001513685,2,AUTOMOVIL,SIN INFORMACION,SIN INFORMACION,S
2,14283609,A001514310,1,CAMIONETA,PARTICULAR,SIN INFORMACION,N
3,14283859,A001514317,1,MOTOCICLETA,PARTICULAR,SIN INFORMACION,N
4,14283860,A001514317,2,CAMIONETA,PARTICULAR,SIN INFORMACION,N


In [11]:
##################################################
##### Update database #####
##################################################

# https://www.geeksforgeeks.org/how-to-insert-a-pandas-dataframe-to-an-existing-postgresql-table/
def execute_values(conn, df, table):
    """
    This function inserts a DataFrame into a given table from a given database.

    Args:
        conn: database connection.
        df: DataFrame to be inserted.
        table: table where the df is inserted. 

    Returns:
        The table is updated with info from df.
    """

    tuples = [tuple(x) for x in df.to_numpy()]
  
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the DataFrame is inserted")
    cursor.close()

# We connect to the database accidents_smb
db_conn = psycopg2.connect(
   database = "accidents_smb", user = "dev", password = "dev", host = "127.0.0.1", port = "5432"
)

# We update the tables from accidents_smb with the info pulled from the GIS web service and further processed to keep
# consistency with the structure defined in the initial ETL process
execute_values(db_conn, accidents_df, "siniestros")
execute_values(db_conn, injured_people_df, "conheridos")
execute_values(db_conn, killed_people_df, "confallecidos")
execute_values(db_conn, causes_df, "causas")
execute_values(db_conn, actors_df, "actores")
execute_values(db_conn, vehicles_df, "vehiculos")

# We close the connection to accidents_smb
db_conn.close()

the DataFrame is inserted
the DataFrame is inserted
the DataFrame is inserted
the DataFrame is inserted
the DataFrame is inserted
the DataFrame is inserted
